# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.drop(cities_df.columns[[0]], axis=1, inplace=True)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,carnarvon,-24.87,113.63,66.67,82,0,8.90,AU,1666475539
1,albany,42.60,-73.97,66.85,61,0,2.55,US,1666475308
2,kapaa,22.08,-159.32,84.52,78,100,16.11,US,1666475539
3,asau,46.43,26.40,51.28,56,100,6.40,RO,1666475540
4,conway,35.09,-92.44,82.20,46,0,19.57,US,1666475310
...,...,...,...,...,...,...,...,...,...
554,manzanillo,20.34,-77.12,79.56,81,93,5.35,CU,1666475708
555,isla vista,34.41,-119.86,73.26,72,61,5.01,US,1666475709
556,newcastle,-32.93,151.78,66.81,96,100,1.01,AU,1666475575
557,acari,-15.43,-74.62,72.19,59,0,6.69,PE,1666475709


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity_level = cities_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_level, 
                                 dissipating=False,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities_nice_weather = cities_df[(cities_df['Max Temp'] >= 65) &
                                (cities_df['Max Temp'] <= 80) &
                                (cities_df['Wind Speed'] <= 3) &
                                (cities_df['Cloudiness'] <= 10)].dropna().reset_index(drop=True)

hotel_df = cities_nice_weather.loc[:, ['City', 'Country']]
hotel_df['Hotel Name'] = ""
hotel_df['Lat'] = ""
hotel_df['Lng'] = ""

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "type": "lodging",
    "key": g_key
}

for index, row in cities_nice_weather.iterrows():

    params['location'] = str(row['Lat']) + "," + str(row['Lng'])

    print(f"Retrieving Result {index} for {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = float(results[0]['geometry']['location']['lat'])
        hotel_df.loc[index, 'Lng'] = float(results[0]['geometry']['location']['lng'])
        
    except (IndexError, KeyError):
        print("Missing field/result... deleting city.")
        hotel_df = hotel_df.drop(index)
        
    print("------------")
          
hotel_df

Retrieving Result 0 for albany.
------------
Retrieving Result 1 for yazman.
------------
Retrieving Result 2 for najran.
------------
Retrieving Result 3 for tiberias.
------------
Retrieving Result 4 for ambilobe.
------------
Retrieving Result 5 for sheffield lake.
------------
Retrieving Result 6 for martinsville.
------------
Retrieving Result 7 for innisfail.
------------
Retrieving Result 8 for banat.
------------


,City,Country,Hotel Name,Lat,Lng
0,albany,US,Hilton Garden Inn Albany Medical Center,42.6523,-73.7762
1,yazman,PK,Malik House,29.1217,71.7421
2,najran,SA,Daniel Hotel Apartments 1 Najran,17.5371,44.2148
3,tiberias,IL,Aviv Hostel,32.7832,35.5422
4,ambilobe,MG,Iharana Bush Camp,-12.9305,49.1568
5,sheffield lake,US,Fairfield Inn & Suites by Marriott Cleveland Avon,41.4666,-82.0577
6,martinsville,US,Hampton Inn Martinsville,36.7051,-79.9104
7,innisfail,AU,Castaways Resort & Spa Mission Beach,-17.8719,146.106
8,banat,IN,"Pathak Bhawan ,Rishto Ka Sansar",29.4507,77.3192


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))